# Inserting ISS Data To MySQL with SQLAlchemy

## Collecting data from the API

In [6]:
import keys

In [39]:
import requests
url =  "http://api.open-notify.org/iss-now.json"
response = requests.get(url)
json_response = response.json()
json_response

{'message': 'success',
 'timestamp': 1672911400,
 'iss_position': {'latitude': '37.8174', 'longitude': '-51.9424'}}

Extract only the pieces of info we need and store them in a dictionary

In [40]:
data_to_insert = {"latitude":[json_response["iss_position"]["latitude"]],
                  "longitude":[json_response["iss_position"]["longitude"]],
                  "iss_timestamp":[json_response["timestamp"]]}
data_to_insert

{'latitude': ['37.8174'],
 'longitude': ['-51.9424'],
 'iss_timestamp': [1672911400]}

Create a pandas Dataframe with the same columns as the SQL table we want to have in our database

In [41]:
import pandas as pd

iss_df = pd.DataFrame.from_dict(data_to_insert)
iss_df

,latitude,longitude,iss_timestamp
0,37.8174,-51.9424,1672911400


Timestamp from UNIX to proper datetime format

In [42]:
iss_df.iss_timestamp = pd.to_datetime(iss_df.iss_timestamp,unit='s')
iss_df

,latitude,longitude,iss_timestamp
0,37.8174,-51.9424,2023-01-05 09:36:40


In [43]:
data_to_insert = {"iss_timestamp":[json_response["timestamp"]],
                  "activity_status":["active"]}
status_df = pd.DataFrame.from_dict(data_to_insert)
status_df.iss_timestamp = pd.to_datetime(status_df.iss_timestamp,unit='s')
status_df

,iss_timestamp,activity_status
0,2023-01-05 09:36:40,active


# SQLAlchemy

In [29]:
# pip install sqlalchemy 
import sqlalchemy # install if needed

Specify MySQL connection.

Before this part you should have already created a schema (database) `sql_workshop` on you local mySql server with coresponding tables `iss_logs` and `iss_status`.

Data that we want to insert into a ddatabase should follow the same structure as a table in a database.

In [35]:
schema="sql_workshop"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password=keys.password # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

Use pandas method `to_sql` with the argument `if_exists=append` to create the table (only the first time we run it) and insert the new rows into it.

In [34]:
status_df

,iss_timestamp,activity_status
0,2023-01-05 09:03:24,active


In [44]:
status_df.to_sql('iss_status',con=con,if_exists='append',index=False)

1

Check on MySQLWorkbench that a new table `iss_status` exists within the `iss_workshop` database, and that a new row has been inserted on it. If you run the whole notebook again, another row should appear there.

We can now populate the other table `iss_logs` by giving a fiction status active to the same timestamp.

In [37]:
iss_df

,latitude,longitude,iss_timestamp
0,-0.4128,174.1117,2023-01-05 09:03:24


In [45]:
iss_df.to_sql('iss_logs',         # 'iss_logs'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False)        # index = False -> will not send index column to database                                          

1

In [46]:
table_df = pd.read_sql_table(
    'iss_logs',
    con=con
    )

In [47]:
table_df

,id,latitude,longitude,iss_timestamp
0,2,-0.4128,174.1120,2023-01-05 09:03:24
1,3,37.8174,-51.9424,2023-01-05 09:36:40
